In [39]:
from unittest.mock import patch
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
import httpx
from openai import RateLimitError

openai_api_key = 'EMPTY'
openai_api_base = 'http://localhost:11434/v1'
openai_llm = ChatOpenAI(openai_api_key=openai_api_key, openai_api_base=openai_api_base, temperature=0, max_tokens=256, max_retries=0)
qwen_llm = ChatOllama(model="qwen:1.8b")
llm = openai_llm.with_fallbacks([qwen_llm])

# 创建一个 RateLimitError 对象
response = httpx.Response(
    status_code=429,                                                       # 429: Too Many Requests 错误码
    request=httpx.Request('GET', 'https://api.openai.com/v1/completions')  # 模拟一个请求对象
)

body = {
    'error': {
        'message': 'Rate limit exceeded.',
        'type': 'rate_limit_error',
        'param': None,
        'code': 'rate_limit_error'
    }
}

error = RateLimitError(message="Rate limit exceeded.", response=response, body=body)
# print(error)

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

遇到错误


In [40]:
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("你是谁？"))
    except RateLimitError:
        print("遇到错误")

content='我是来自阿里云的大规模语言模型，我叫通义千问。作为一个AI助手，我的目标是理解和生成人类自然语言，如文字、对话等。\n\n通义千问具有以下优势：\n\n1. **强大的学习能力**：通义千问能够从海量文本数据中学习和理解语言的语法结构、词汇意义以及语用功能。\n\n2. **跨语言沟通的能力**：通义千问能够理解和生成多种语言，包括但不限于英语（US）、法语（FR）、德语（DE）等。这些能力不仅能让通义千问在多语言环境下进行准确和流畅的语言交流，还能为用户提供更加个性化的服务体验。\n\n总之，通义千问作为阿里云的超大规模语言模型，具备强大的学习能力和跨语言沟通的能力，能够在多种语言环境下进行准确和流畅的语言交流，为用户提供更加个性化的服务体验。' additional_kwargs={} response_metadata={'model': 'qwen:1.8b', 'created_at': '2025-02-20T08:29:12.254154811Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 49461756157, 'load_duration': 2081415844, 'prompt_eval_count': 11, 'prompt_eval_duration': 275000000, 'eval_count': 185, 'eval_duration': 47104000000} id='run-bc485ca3-4330-47a9-a296-28a8b4931dec-0'
